In [1]:
import os
import json
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import random
import numpy as np
from collections import defaultdict
import scipy.stats as st

import networkx as nx
import gmatch4py as gm

import sys
sys.path.append("..\\Git\\complex_stylized_supply_chain_model_generator")

from complex_toy_model_graph_generator import ComplexSimModelGraph
from structure_model_composer.set_locations_of_structures import plot_graph_locations_with_pos

<frozen importlib._bootstrap>:228: UserWarning: To obtain optimal results install the Cython 'munkres' module at  https://github.com/jfrelinger/cython-munkres-wrapper


In [2]:
data={}
x={}
y={}

In [18]:
# #upload all percentages
# technique = "bo"
# sparseness = "bias"
# directory = f"./{technique}_{sparseness}"
# #Over folders
# for folder in os.listdir(directory):
#     if not folder.startswith('.'):
#         new_dir = directory+"/"+folder
#         if os.path.isdir(new_dir):
#             for filename in os.listdir(new_dir):
#                 if filename.endswith(".pkl"):
#                     f = new_dir+"/"+filename

#                     key = filename.split('_')[:2]
#                     key = "_".join(key)

#                     with open(f, "rb") as file:
#                          info = pickle.load(file)
                        
#                     seed = info["Model_info"]["parameters"]["seed"]
                    
#                     try:
#                         data[seed][key] = info
#                     except KeyError:
#                         data[seed] = {key: info}

# for k, v in data.items():
#     data[k] = dict(sorted(data[k].items(), key=lambda item: int(item[0].split('_')[1])))

In [3]:
# results for interaction
technique = "bo"
directory = f"./interaction/{technique}"
#Over folders
for folder in os.listdir(directory):
    if not folder.startswith('.'):
        new_dir = directory+"/"+folder
        technique = folder.split("_")[-1]
        percentage = folder.split("_")[-3]
        if os.path.isdir(new_dir):
            for filename in os.listdir(new_dir):
                if filename.endswith(".pkl"):
                    f = new_dir+"/"+filename

                    key = f"{technique}_{percentage}"

                    with open(f, "rb") as file:
                         info = pickle.load(file)
                        
                    seed = info["Model_info"]["parameters"]["seed"]
                    
                    try:
                        data[seed][key] = info
                    except KeyError:
                        data[seed] = {key: info}

for k, v in data.items():
    data[k] = dict(sorted(data[k].items(), key=lambda item: int(item[0].split('_')[1])))

In [4]:
for k, v in data.items():
    print(k, data[k].keys())

1 dict_keys(['bo_28', 'bo_82', 'bo_88'])
6 dict_keys(['bo_28', 'bo_82', 'bo_88'])
11 dict_keys(['bo_28', 'bo_82', 'bo_88'])
16 dict_keys(['bo_28', 'bo_82', 'bo_88'])
21 dict_keys(['bo_28', 'bo_82', 'bo_88'])
26 dict_keys(['bo_28', 'bo_82', 'bo_88'])


# Calculate Quality of Fit

In [5]:
#Basic parameters
with open(r"../../complex_stylized_supply_chain_model_generator/data/location_model_hpc_40000_cnhk_usa_airsea.pkl", "rb") as f:
    decision_variables = pickle.load(f)

with open(r"../../complex_stylized_supply_chain_model_generator/data/Ground_Truth_Graph_Topology_DF_CNHK_USA.pkl", "rb") as f:
    ground_truth_topology = pickle.load(f)

In [6]:
gt_graph = ground_truth_topology["graph"][0]

for seed, attr in data.items():
    for perc, res in attr.items():
        sol_idx = data[seed][perc]["Results"]["graph_structure"][0]
        sol_obj_value = data[seed][perc]["Results"]["min_distance"][0]
        
        sol_topology = decision_variables[round(sol_idx)]
        sol_graph = sol_topology["graph"]
        
        # combine ground truth and solutions
        column_names = ["edges", "nodes", "betweenness", "degree_centrality", "closeness_centrality"]
        df_sol_topology = pd.DataFrame.from_dict(sol_topology, orient="index").T[column_names].rename(index={0: 'solution'})
        df_gt_topology = ground_truth_topology[column_names].rename(index={0: 'ground_truth'})
        
        df_combined = pd.concat([df_gt_topology, df_sol_topology])
        
        # calculate differences
        
        diff_values = df_combined.loc['ground_truth'] - df_combined.loc['solution']
        df_diff = pd.DataFrame([diff_values], index=['diff'])
        
        df_combined_diff = pd.concat([df_combined, df_diff])
        
        # calculate graph edit distance
        ged=gm.GraphEditDistance(1,1,1,1) # all edit costs are equal to 1
        result=ged.compare([gt_graph, sol_graph],None) 
        
        distance = ged.distance(result)[1][0]
        
        # add to dictionary data
        data[seed][perc]["Quality_of_Fit"] = {"features": df_combined_diff, "graph_edit_distance": distance, "abs_dist": np.sum(result)}



In [7]:
# file_name = f"./{technique}_{sparseness}/final_{technique}_{sparseness}_all_seeds.pkl"
# with open(file_name, "wb") as output:
#     pickle.dump(data, output)

file_name = directory + f"/final_interaction_{technique}_all_seeds.pkl"
with open(file_name, "wb") as output:
    pickle.dump(data, output)